In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
train = pd.read_csv('trainMaster.csv')

# Target

In [3]:
x = train['comment_text']

y1 = train['toxic']
y2 = train['severe_toxic']
y3 = train['obscene']
y4 = train['threat']
y5 = train['insult']
y6 = train['identity_hate']
y7 = train['clean']

datay = [y1, y2, y3, y4, y5, y6, y7]

# Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
dSplit = {}
for j in range(1,8):
        X_train, X_test, dSplit['y{}_train'.format(j)], dSplit['y{}_test'.format(j)] = train_test_split(x, datay[j-1], test_size=0.3, random_state=101)

In [6]:
print('OwO')

OwO


# K Fold

In [7]:
# from sklearn.model_selection import KFold

In [8]:
# K = 5
# kf = KFold(n_splits=K, shuffle=True, random_state = 42)

In [9]:
# dFold = {}
# for i in range(1,8):
#     dFold['Hasilfold{}'.format(i)] = kf.split(x,datay[i-1])

# Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer


# Analyzer for Pipeline

In [11]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

# stemmer = SnowballStemmer("english")
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence

def process_normal(text):
    return [word for word in text.split() if word.lower() not in stop]

def process_nocase(text):
    return [word.lower() for word in text.split() if word.lower() not in stop]

def process_nopunc(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stop]

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

# Pipeline

In [12]:
listAnalyzer = [process_normal, process_nocase,process_nopunc,process_nocasepunc]
nama = ['normal','noCase','noPunc','noCasePunc']
dPP = {}
dPP2 = {}

# buat analyzer
for i in range(0,4): 
#     buat n_gram
    for j in range(1,4):
        dPP['pp_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('classifier', LogisticRegression()),
        ])

        # With TFIDF
        dPP2['pp2_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('tfidf', TfidfTransformer()),
            ('classifier', LogisticRegression()),
        ])

In [13]:
print('OwO')

OwO


# Predict

In [14]:
ppKey = list(dPP.keys())
pp2Key = list(dPP2.keys())

dPredict_train = {}
dPredict_test = {}
dPredictProba_train = {}
dPredictProba_test = {}

dPredict2_train = {}
dPredict2_test = {}
dPredict2Proba_train = {}
dPredict2Proba_test = {}

for item, item2 in zip(ppKey, pp2Key):
    for i in range(1,8):
#         Pipeline
        dPP[item].fit(X_train, dSplit['y{}_train'.format(i)])
        
        dPredict_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict(X_train)
        dPredictProba_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_train)
        
        dPredict_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict(X_test)
        dPredictProba_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_test)
        
#         Pipeline2
        dPP2[item2].fit(X_train, dSplit['y{}_train'.format(i)])
    
        dPredict2_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_train)
        dPredict2Proba_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_train)
        
        dPredict2_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_test)
        dPredict2Proba_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_test)

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [15]:
print('OwO')

OwO


# Report

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

In [17]:
def calc_train_error(X_train, y_train, predictTrain, predictProbaTrain):
    matt = matthews_corrcoef(y_train, predictTrain)
    f1 = f1_score(y_train, predictTrain, average = 'macro')
    report = classification_report(y_train, predictTrain)
    roc_auc = roc_auc_score(y_train, predictProbaTrain[:,1])
    accuracy = accuracy_score(y_train, predictTrain)
    confMatrix = confusion_matrix(y_train, predictTrain)
    logloss = log_loss(y_train, predictProbaTrain)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_validation_error(X_test, y_test, predictTest, predictProbaTest):
    matt = matthews_corrcoef(y_test, predictTest)
    f1 = f1_score(y_test, predictTest, average = 'macro')
    report = classification_report(y_test, predictTest)
    roc_auc = roc_auc_score(y_test, predictProbaTest[:,1])
    accuracy = accuracy_score(y_test, predictTest)
    confMatrix = confusion_matrix(y_test, predictTest)
    logloss = log_loss(y_test, predictProbaTest)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_metrics(X_train, y_train, X_test, y_test, predictTrain, predictProbaTrain, predictTest, predictProbaTest):
    train_error = calc_train_error(X_train, y_train, predictTrain, predictProbaTrain)
    validation_error = calc_validation_error(X_test, y_test, predictTest, predictProbaTest)
    return train_error, validation_error

## Toxic

In [18]:
dToxic_train = {}
dToxic_test = {}
dToxic2_train = {}
dToxic2_test = {}

for item in ppKey:
    dToxic_train['hasil_train_{}'.format(item)], dToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y1_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y1_{}'.format(item)])

for item2 in pp2Key:
    dToxic2_train['hasil_train_{}'.format(item2)], dToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict2_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y1_{}'.format(item2)])

In [19]:
dToxic_train_key = list(dToxic_train.keys())
dToxic_test_key = list(dToxic_test.keys())
listItem = []

for train, test in zip(dToxic_train_key, dToxic_test_key):
    listItem.append(['log_{}'.format(test),
                     dToxic_train[train]['accuracy'],
                     dToxic_test[test]['accuracy'],
                     dToxic_train[train]['roc'],
                     dToxic_test[test]['roc'],
                     dToxic_train[train]['f1'],
                     dToxic_test[test]['f1'],
                     dToxic_train[train]['matthew'],
                     dToxic_test[test]['matthew'],
                     dToxic_train[train]['logloss'],
                     dToxic_test[test]['logloss'],])


dfToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.955353,0.943412,0.978711,0.936664,0.841542,0.795874,0.712022,0.624404,0.117631,0.168191
1,log_hasil_test_pp_normal_ngram_2,0.955353,0.943412,0.978711,0.936664,0.841542,0.795874,0.712022,0.624404,0.117631,0.168191
2,log_hasil_test_pp_normal_ngram_3,0.955353,0.943412,0.978711,0.936664,0.841542,0.795874,0.712022,0.624404,0.117631,0.168191
3,log_hasil_test_pp_noCase_ngram_1,0.953867,0.945208,0.974680,0.939702,0.836479,0.804605,0.701143,0.638887,0.122580,0.165429
4,log_hasil_test_pp_noCase_ngram_2,0.953867,0.945208,0.974680,0.939702,0.836479,0.804605,0.701143,0.638887,0.122580,0.165429
5,log_hasil_test_pp_noCase_ngram_3,0.953867,0.945208,0.974680,0.939702,0.836479,0.804605,0.701143,0.638887,0.122580,0.165429
6,log_hasil_test_pp_noPunc_ngram_1,0.959095,0.950054,0.980765,0.949621,0.859529,0.828287,0.739478,0.677335,0.110113,0.155823
7,log_hasil_test_pp_noPunc_ngram_2,0.959095,0.950054,0.980765,0.949621,0.859529,0.828287,0.739478,0.677335,0.110113,0.155823
8,log_hasil_test_pp_noPunc_ngram_3,0.959095,0.950054,0.980765,0.949621,0.859529,0.828287,0.739478,0.677335,0.110113,0.155823
9,log_hasil_test_pp_noCasePunc_ngram_1,0.956392,0.951078,0.975096,0.951024,0.848945,0.830927,0.720095,0.684117,0.117898,0.153365


In [20]:
dToxic2_train_key = list(dToxic2_train.keys())
dToxic2_test_key = list(dToxic2_test.keys())
listItem = []

for train, test in zip(dToxic2_train_key, dToxic2_test_key):
    listItem.append(['log_{}'.format(test),
                     dToxic2_train[train]['accuracy'],
                     dToxic2_test[test]['accuracy'],
                     dToxic2_train[train]['roc'],
                     dToxic2_test[test]['roc'],
                     dToxic2_train[train]['f1'],
                     dToxic2_test[test]['f1'],
                     dToxic2_train[train]['matthew'],
                     dToxic2_test[test]['matthew'],
                     dToxic2_train[train]['logloss'],
                     dToxic2_test[test]['logloss'],])


dfToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.950358,0.945480,0.983230,0.951962,0.815256,0.798442,0.674406,0.638682,0.125992,0.148538
1,log_hasil_test_pp2_normal_ngram_2,0.950358,0.945480,0.983230,0.951962,0.815256,0.798442,0.674406,0.638682,0.125992,0.148538
2,log_hasil_test_pp2_normal_ngram_3,0.950358,0.945480,0.983230,0.951962,0.815256,0.798442,0.674406,0.638682,0.125992,0.148538
3,log_hasil_test_pp2_noCase_ngram_1,0.953160,0.949198,0.983983,0.956075,0.829741,0.816629,0.695665,0.667947,0.120354,0.141314
4,log_hasil_test_pp2_noCase_ngram_2,0.953160,0.949198,0.983983,0.956075,0.829741,0.816629,0.695665,0.667947,0.120354,0.141314
5,log_hasil_test_pp2_noCase_ngram_3,0.953160,0.949198,0.983983,0.956075,0.829741,0.816629,0.695665,0.667947,0.120354,0.141314
6,log_hasil_test_pp2_noPunc_ngram_1,0.956410,0.952206,0.985786,0.965677,0.845074,0.830778,0.719782,0.690905,0.112753,0.130338
7,log_hasil_test_pp2_noPunc_ngram_2,0.956410,0.952206,0.985786,0.965677,0.845074,0.830778,0.719782,0.690905,0.112753,0.130338
8,log_hasil_test_pp2_noPunc_ngram_3,0.956410,0.952206,0.985786,0.965677,0.845074,0.830778,0.719782,0.690905,0.112753,0.130338
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.958818,0.955882,0.986050,0.968238,0.856847,0.848085,0.737291,0.718353,0.107278,0.123039


In [21]:
# print(confusion_matrix(dSplit['y1_test'], predictions))
# print('\n')
# print(classification_report(dSplit['y1_test'], predictions))

## Severe Toxic

In [22]:
dSToxic_train = {}
dSToxic_test = {}
dSToxic2_train = {}
dSToxic2_test = {}

for item in ppKey:
    dSToxic_train['hasil_train_{}'.format(item)], dSToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y2_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y2_{}'.format(item)])

for item2 in pp2Key:
    dSToxic2_train['hasil_train_{}'.format(item2)], dSToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict2_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y2_{}'.format(item2)])

In [23]:
dSToxic_train_key = list(dSToxic_train.keys())
dSToxic_test_key = list(dSToxic_test.keys())
listItem = []

for train, test in zip(dSToxic_train_key, dSToxic_test_key):
    listItem.append(['log_{}'.format(test),
                     dSToxic_train[train]['accuracy'],
                     dSToxic_test[test]['accuracy'],
                     dSToxic_train[train]['roc'],
                     dSToxic_test[test]['roc'],
                     dSToxic_train[train]['f1'],
                     dSToxic_test[test]['f1'],
                     dSToxic_train[train]['matthew'],
                     dSToxic_test[test]['matthew'],
                     dSToxic_train[train]['logloss'],
                     dSToxic_test[test]['logloss'],])


dfSToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.991021,0.989660,0.973707,0.953004,0.607470,0.580030,0.304780,0.227264,0.034967,0.048376
1,log_hasil_test_pp_normal_ngram_2,0.991021,0.989660,0.973707,0.953004,0.607470,0.580030,0.304780,0.227264,0.034967,0.048376
2,log_hasil_test_pp_normal_ngram_3,0.991021,0.989660,0.973707,0.953004,0.607470,0.580030,0.304780,0.227264,0.034967,0.048376
3,log_hasil_test_pp_noCase_ngram_1,0.991074,0.989597,0.973859,0.944716,0.620960,0.589186,0.321954,0.238059,0.033907,0.049215
4,log_hasil_test_pp_noCase_ngram_2,0.991074,0.989597,0.973859,0.944716,0.620960,0.589186,0.321954,0.238059,0.033907,0.049215
5,log_hasil_test_pp_noCase_ngram_3,0.991074,0.989597,0.973859,0.944716,0.620960,0.589186,0.321954,0.238059,0.033907,0.049215
6,log_hasil_test_pp_noPunc_ngram_1,0.991083,0.989723,0.979739,0.965423,0.618184,0.595453,0.319848,0.252972,0.034278,0.047314
7,log_hasil_test_pp_noPunc_ngram_2,0.991083,0.989723,0.979739,0.965423,0.618184,0.595453,0.319848,0.252972,0.034278,0.047314
8,log_hasil_test_pp_noPunc_ngram_3,0.991083,0.989723,0.979739,0.965423,0.618184,0.595453,0.319848,0.252972,0.034278,0.047314
9,log_hasil_test_pp_noCasePunc_ngram_1,0.991038,0.989764,0.979098,0.960080,0.625073,0.600984,0.323886,0.262776,0.033341,0.047229


In [24]:
dSToxic2_train_key = list(dSToxic2_train.keys())
dSToxic2_test_key = list(dSToxic2_test.keys())
listItem = []

for train, test in zip(dSToxic2_train_key, dSToxic2_test_key):
    listItem.append(['log_{}'.format(test),
                     dSToxic2_train[train]['accuracy'],
                     dSToxic2_test[test]['accuracy'],
                     dSToxic2_train[train]['roc'],
                     dSToxic2_test[test]['roc'],
                     dSToxic2_train[train]['f1'],
                     dSToxic2_test[test]['f1'],
                     dSToxic2_train[train]['matthew'],
                     dSToxic2_test[test]['matthew'],
                     dSToxic2_train[train]['logloss'],
                     dSToxic2_test[test]['logloss'],])


dfSToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.991110,0.989764,0.992669,0.981265,0.626410,0.627333,0.329878,0.300270,0.026142,0.031228
1,log_hasil_test_pp2_normal_ngram_2,0.991110,0.989764,0.992669,0.981265,0.626410,0.627333,0.329878,0.300270,0.026142,0.031228
2,log_hasil_test_pp2_normal_ngram_3,0.991110,0.989764,0.992669,0.981265,0.626410,0.627333,0.329878,0.300270,0.026142,0.031228
3,log_hasil_test_pp2_noCase_ngram_1,0.991226,0.990057,0.992428,0.982581,0.651747,0.661337,0.363480,0.357176,0.025389,0.030498
4,log_hasil_test_pp2_noCase_ngram_2,0.991226,0.990057,0.992428,0.982581,0.651747,0.661337,0.363480,0.357176,0.025389,0.030498
5,log_hasil_test_pp2_noCase_ngram_3,0.991226,0.990057,0.992428,0.982581,0.651747,0.661337,0.363480,0.357176,0.025389,0.030498
6,log_hasil_test_pp2_noPunc_ngram_1,0.991182,0.990015,0.993237,0.983526,0.649725,0.642887,0.358973,0.330353,0.024893,0.029475
7,log_hasil_test_pp2_noPunc_ngram_2,0.991182,0.990015,0.993237,0.983526,0.649725,0.642887,0.358973,0.330353,0.024893,0.029475
8,log_hasil_test_pp2_noPunc_ngram_3,0.991182,0.990015,0.993237,0.983526,0.649725,0.642887,0.358973,0.330353,0.024893,0.029475
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.991244,0.990370,0.992967,0.985295,0.669167,0.676990,0.384452,0.387388,0.024297,0.029071


## Obscene

In [25]:
dObscene_train = {}
dObscene_test = {}
dObscene2_train = {}
dObscene2_test = {}

for item in ppKey:
    dObscene_train['hasil_train_{}'.format(item)], dObscene_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y3_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y3_{}'.format(item)])

for item2 in pp2Key:
    dObscene2_train['hasil_train_{}'.format(item2)], dObscene2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict2_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y3_{}'.format(item2)])

In [26]:
dObscene_train_key = list(dObscene_train.keys())
dObscene_test_key = list(dObscene_test.keys())
listItem = []

for train, test in zip(dObscene_train_key, dObscene_test_key):
    listItem.append(['log_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfObsceneResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObsceneResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.975452,0.970839,0.983186,0.947062,0.847928,0.822223,0.721218,0.669531,0.074023,0.112047
1,log_hasil_test_pp_normal_ngram_2,0.975452,0.970839,0.983186,0.947062,0.847928,0.822223,0.721218,0.669531,0.074023,0.112047
2,log_hasil_test_pp_normal_ngram_3,0.975452,0.970839,0.983186,0.947062,0.847928,0.822223,0.721218,0.669531,0.074023,0.112047
3,log_hasil_test_pp_noCase_ngram_1,0.971683,0.969418,0.967432,0.944221,0.816804,0.807548,0.669785,0.648301,0.089668,0.116235
4,log_hasil_test_pp_noCase_ngram_2,0.971683,0.969418,0.967432,0.944221,0.816804,0.807548,0.669785,0.648301,0.089668,0.116235
5,log_hasil_test_pp_noCase_ngram_3,0.971683,0.969418,0.967432,0.944221,0.816804,0.807548,0.669785,0.648301,0.089668,0.116235
6,log_hasil_test_pp_noPunc_ngram_1,0.978290,0.974411,0.987414,0.963517,0.871338,0.850959,0.758486,0.717738,0.066691,0.101503
7,log_hasil_test_pp_noPunc_ngram_2,0.978290,0.974411,0.987414,0.963517,0.871338,0.850959,0.758486,0.717738,0.066691,0.101503
8,log_hasil_test_pp_noPunc_ngram_3,0.978290,0.974411,0.987414,0.963517,0.871338,0.850959,0.758486,0.717738,0.066691,0.101503
9,log_hasil_test_pp_noCasePunc_ngram_1,0.973903,0.971695,0.976662,0.963330,0.836992,0.827644,0.700919,0.680431,0.081156,0.103624


In [27]:
dObscene2_train_key = list(dObscene2_train.keys())
dObscene2_test_key = list(dObscene2_test.keys())
listItem = []

for train, test in zip(dObscene2_train_key, dObscene2_test_key):
    listItem.append(['log_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfObscene2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObscene2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.972945,0.971946,0.989775,0.963925,0.824014,0.824512,0.686867,0.681866,0.074250,0.087744
1,log_hasil_test_pp2_normal_ngram_2,0.972945,0.971946,0.989775,0.963925,0.824014,0.824512,0.686867,0.681866,0.074250,0.087744
2,log_hasil_test_pp2_normal_ngram_3,0.972945,0.971946,0.989775,0.963925,0.824014,0.824512,0.686867,0.681866,0.074250,0.087744
3,log_hasil_test_pp2_noCase_ngram_1,0.975318,0.973241,0.990860,0.969614,0.845001,0.837511,0.719087,0.700067,0.068720,0.081434
4,log_hasil_test_pp2_noCase_ngram_2,0.975318,0.973241,0.990860,0.969614,0.845001,0.837511,0.719087,0.700067,0.068720,0.081434
5,log_hasil_test_pp2_noCase_ngram_3,0.975318,0.973241,0.990860,0.969614,0.845001,0.837511,0.719087,0.700067,0.068720,0.081434
6,log_hasil_test_pp2_noPunc_ngram_1,0.976517,0.974891,0.993329,0.979048,0.855219,0.850067,0.734955,0.721568,0.064023,0.074870
7,log_hasil_test_pp2_noPunc_ngram_2,0.976517,0.974891,0.993329,0.979048,0.855219,0.850067,0.734955,0.721568,0.064023,0.074870
8,log_hasil_test_pp2_noPunc_ngram_3,0.976517,0.974891,0.993329,0.979048,0.855219,0.850067,0.734955,0.721568,0.064023,0.074870
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.978496,0.976542,0.993954,0.982842,0.871401,0.864211,0.760580,0.743485,0.058683,0.068160


## Threat

In [28]:
dThreat_train = {}
dThreat_test = {}
dThreat2_train = {}
dThreat2_test = {}

for item in ppKey:
    dThreat_train['hasil_train_{}'.format(item)], dThreat_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y4_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y4_{}'.format(item)])

for item2 in pp2Key:
    dThreat2_train['hasil_train_{}'.format(item2)], dThreat2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict2_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y4_{}'.format(item2)])

In [29]:
dThreat_train_key = list(dThreat_train.keys())
dThreat_test_key = list(dThreat_test.keys())
listItem = []

for train, test in zip(dThreat_train_key, dThreat_test_key):
    listItem.append(['log_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfThreatResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreatResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.975452,0.970839,0.983186,0.947062,0.847928,0.822223,0.721218,0.669531,0.074023,0.112047
1,log_hasil_test_pp_normal_ngram_2,0.975452,0.970839,0.983186,0.947062,0.847928,0.822223,0.721218,0.669531,0.074023,0.112047
2,log_hasil_test_pp_normal_ngram_3,0.975452,0.970839,0.983186,0.947062,0.847928,0.822223,0.721218,0.669531,0.074023,0.112047
3,log_hasil_test_pp_noCase_ngram_1,0.971683,0.969418,0.967432,0.944221,0.816804,0.807548,0.669785,0.648301,0.089668,0.116235
4,log_hasil_test_pp_noCase_ngram_2,0.971683,0.969418,0.967432,0.944221,0.816804,0.807548,0.669785,0.648301,0.089668,0.116235
5,log_hasil_test_pp_noCase_ngram_3,0.971683,0.969418,0.967432,0.944221,0.816804,0.807548,0.669785,0.648301,0.089668,0.116235
6,log_hasil_test_pp_noPunc_ngram_1,0.978290,0.974411,0.987414,0.963517,0.871338,0.850959,0.758486,0.717738,0.066691,0.101503
7,log_hasil_test_pp_noPunc_ngram_2,0.978290,0.974411,0.987414,0.963517,0.871338,0.850959,0.758486,0.717738,0.066691,0.101503
8,log_hasil_test_pp_noPunc_ngram_3,0.978290,0.974411,0.987414,0.963517,0.871338,0.850959,0.758486,0.717738,0.066691,0.101503
9,log_hasil_test_pp_noCasePunc_ngram_1,0.973903,0.971695,0.976662,0.963330,0.836992,0.827644,0.700919,0.680431,0.081156,0.103624


In [30]:
dThreat2_train_key = list(dThreat2_train.keys())
dThreat2_test_key = list(dThreat2_test.keys())
listItem = []

for train, test in zip(dThreat2_train_key, dThreat2_test_key):
    listItem.append(['log_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfThreat2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreat2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.972945,0.971946,0.989775,0.963925,0.824014,0.824512,0.686867,0.681866,0.074250,0.087744
1,log_hasil_test_pp2_normal_ngram_2,0.972945,0.971946,0.989775,0.963925,0.824014,0.824512,0.686867,0.681866,0.074250,0.087744
2,log_hasil_test_pp2_normal_ngram_3,0.972945,0.971946,0.989775,0.963925,0.824014,0.824512,0.686867,0.681866,0.074250,0.087744
3,log_hasil_test_pp2_noCase_ngram_1,0.975318,0.973241,0.990860,0.969614,0.845001,0.837511,0.719087,0.700067,0.068720,0.081434
4,log_hasil_test_pp2_noCase_ngram_2,0.975318,0.973241,0.990860,0.969614,0.845001,0.837511,0.719087,0.700067,0.068720,0.081434
5,log_hasil_test_pp2_noCase_ngram_3,0.975318,0.973241,0.990860,0.969614,0.845001,0.837511,0.719087,0.700067,0.068720,0.081434
6,log_hasil_test_pp2_noPunc_ngram_1,0.976517,0.974891,0.993329,0.979048,0.855219,0.850067,0.734955,0.721568,0.064023,0.074870
7,log_hasil_test_pp2_noPunc_ngram_2,0.976517,0.974891,0.993329,0.979048,0.855219,0.850067,0.734955,0.721568,0.064023,0.074870
8,log_hasil_test_pp2_noPunc_ngram_3,0.976517,0.974891,0.993329,0.979048,0.855219,0.850067,0.734955,0.721568,0.064023,0.074870
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.978496,0.976542,0.993954,0.982842,0.871401,0.864211,0.760580,0.743485,0.058683,0.068160


## Insult

In [31]:
dInsult_train = {}
dInsult_test = {}
dInsult2_train = {}
dInsult2_test = {}

for item in ppKey:
    dInsult_train['hasil_train_{}'.format(item)], dInsult_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y5_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y5_{}'.format(item)])

for item2 in pp2Key:
    dInsult2_train['hasil_train_{}'.format(item2)], dInsult2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict2_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y5_{}'.format(item2)])

In [32]:
dInsult_train_key = list(dInsult_train.keys())
dInsult_test_key = list(dInsult_test.keys())
listItem = []

for train, test in zip(dInsult_train_key, dInsult_test_key):
    listItem.append(['log_{}'.format(test),
                     dInsult_train[train]['accuracy'],
                     dInsult_test[test]['accuracy'],
                     dInsult_train[train]['roc'],
                     dInsult_test[test]['roc'],
                     dInsult_train[train]['f1'],
                     dInsult_test[test]['f1'],
                     dInsult_train[train]['matthew'],
                     dInsult_test[test]['matthew'],
                     dInsult_train[train]['logloss'],
                     dInsult_test[test]['logloss'],])


dfInsultResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsultResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.966123,0.961773,0.967244,0.935990,0.749896,0.718581,0.553168,0.485138,0.095941,0.129962
1,log_hasil_test_pp_normal_ngram_2,0.966123,0.961773,0.967244,0.935990,0.749896,0.718581,0.553168,0.485138,0.095941,0.129962
2,log_hasil_test_pp_normal_ngram_3,0.966123,0.961773,0.967244,0.935990,0.749896,0.718581,0.553168,0.485138,0.095941,0.129962
3,log_hasil_test_pp_noCase_ngram_1,0.963831,0.961522,0.955142,0.931687,0.724079,0.711620,0.510979,0.477009,0.105235,0.134025
4,log_hasil_test_pp_noCase_ngram_2,0.963831,0.961522,0.955142,0.931687,0.724079,0.711620,0.510979,0.477009,0.105235,0.134025
5,log_hasil_test_pp_noCase_ngram_3,0.963831,0.961522,0.955142,0.931687,0.724079,0.711620,0.510979,0.477009,0.105235,0.134025
6,log_hasil_test_pp_noPunc_ngram_1,0.965908,0.963319,0.967737,0.946193,0.749234,0.734512,0.550158,0.513412,0.097475,0.123812
7,log_hasil_test_pp_noPunc_ngram_2,0.965908,0.963319,0.967737,0.946193,0.749234,0.734512,0.550158,0.513412,0.097475,0.123812
8,log_hasil_test_pp_noPunc_ngram_3,0.965908,0.963319,0.967737,0.946193,0.749234,0.734512,0.550158,0.513412,0.097475,0.123812
9,log_hasil_test_pp_noCasePunc_ngram_1,0.964933,0.962964,0.960071,0.942069,0.740755,0.730808,0.533738,0.506946,0.101243,0.127373


In [33]:
dInsult2_train_key = list(dInsult2_train.keys())
dInsult2_test_key = list(dInsult2_test.keys())
listItem = []

for train, test in zip(dInsult2_train_key, dInsult2_test_key):
    listItem.append(['log_{}'.format(test),
                     dInsult2_train[train]['accuracy'],
                     dInsult2_test[test]['accuracy'],
                     dInsult2_train[train]['roc'],
                     dInsult2_test[test]['roc'],
                     dInsult2_train[train]['f1'],
                     dInsult2_test[test]['f1'],
                     dInsult2_train[train]['matthew'],
                     dInsult2_test[test]['matthew'],
                     dInsult2_train[train]['logloss'],
                     dInsult2_test[test]['logloss'],])


dfInsult2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsult2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.968379,0.966139,0.986713,0.960795,0.771984,0.763731,0.591337,0.563348,0.081464,0.096821
1,log_hasil_test_pp2_normal_ngram_2,0.968379,0.966139,0.986713,0.960795,0.771984,0.763731,0.591337,0.563348,0.081464,0.096821
2,log_hasil_test_pp2_normal_ngram_3,0.968379,0.966139,0.986713,0.960795,0.771984,0.763731,0.591337,0.563348,0.081464,0.096821
3,log_hasil_test_pp2_noCase_ngram_1,0.969865,0.966995,0.987037,0.964569,0.790037,0.776629,0.617204,0.581388,0.078197,0.093218
4,log_hasil_test_pp2_noCase_ngram_2,0.969865,0.966995,0.987037,0.964569,0.790037,0.776629,0.617204,0.581388,0.078197,0.093218
5,log_hasil_test_pp2_noCase_ngram_3,0.969865,0.966995,0.987037,0.964569,0.790037,0.776629,0.617204,0.581388,0.078197,0.093218
6,log_hasil_test_pp2_noPunc_ngram_1,0.971065,0.968813,0.988713,0.971702,0.801375,0.792872,0.636200,0.610204,0.074893,0.087517
7,log_hasil_test_pp2_noPunc_ngram_2,0.971065,0.968813,0.988713,0.971702,0.801375,0.792872,0.636200,0.610204,0.074893,0.087517
8,log_hasil_test_pp2_noPunc_ngram_3,0.971065,0.968813,0.988713,0.971702,0.801375,0.792872,0.636200,0.610204,0.074893,0.087517
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.972381,0.969878,0.988645,0.974345,0.816307,0.806231,0.658099,0.630344,0.072249,0.083812


## Identity Hate

In [34]:
dIdHate_train = {}
dIdHate_test = {}
dIdHate2_train = {}
dIdHate2_test = {}

for item in ppKey:
    dIdHate_train['hasil_train_{}'.format(item)], dIdHate_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y6_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y6_{}'.format(item)])

for item2 in pp2Key:
    dIdHate2_train['hasil_train_{}'.format(item2)], dIdHate2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict2_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y6_{}'.format(item2)])

In [35]:
dIdHate_train_key = list(dIdHate_train.keys())
dIdHate_test_key = list(dIdHate_test.keys())
listItem = []

for train, test in zip(dIdHate_train_key, dIdHate_test_key):
    listItem.append(['log_{}'.format(test),
                     dIdHate_train[train]['accuracy'],
                     dIdHate_test[test]['accuracy'],
                     dIdHate_train[train]['roc'],
                     dIdHate_test[test]['roc'],
                     dIdHate_train[train]['f1'],
                     dIdHate_test[test]['f1'],
                     dIdHate_train[train]['matthew'],
                     dIdHate_test[test]['matthew'],
                     dIdHate_train[train]['logloss'],
                     dIdHate_test[test]['logloss'],])


dfIdHateResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHateResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.991961,0.990621,0.974556,0.919278,0.600635,0.547743,0.314818,0.127549,0.030411,0.049632
1,log_hasil_test_pp_normal_ngram_2,0.991961,0.990621,0.974556,0.919278,0.600635,0.547743,0.314818,0.127549,0.030411,0.049632
2,log_hasil_test_pp_normal_ngram_3,0.991961,0.990621,0.974556,0.919278,0.600635,0.547743,0.314818,0.127549,0.030411,0.049632
3,log_hasil_test_pp_noCase_ngram_1,0.991611,0.990579,0.965593,0.903383,0.576200,0.540229,0.254307,0.110555,0.033015,0.052284
4,log_hasil_test_pp_noCase_ngram_2,0.991611,0.990579,0.965593,0.903383,0.576200,0.540229,0.254307,0.110555,0.033015,0.052284
5,log_hasil_test_pp_noCase_ngram_3,0.991611,0.990579,0.965593,0.903383,0.576200,0.540229,0.254307,0.110555,0.033015,0.052284
6,log_hasil_test_pp_noPunc_ngram_1,0.991925,0.990558,0.974779,0.921581,0.609847,0.557938,0.316915,0.146599,0.030680,0.051393
7,log_hasil_test_pp_noPunc_ngram_2,0.991925,0.990558,0.974779,0.921581,0.609847,0.557938,0.316915,0.146599,0.030680,0.051393
8,log_hasil_test_pp_noPunc_ngram_3,0.991925,0.990558,0.974779,0.921581,0.609847,0.557938,0.316915,0.146599,0.030680,0.051393
9,log_hasil_test_pp_noCasePunc_ngram_1,0.991477,0.990830,0.960634,0.904534,0.565131,0.537528,0.227169,0.111732,0.035663,0.052492


In [36]:
dIdHate2_train_key = list(dIdHate2_train.keys())
dIdHate2_test_key = list(dIdHate2_test.keys())
listItem = []

for train, test in zip(dIdHate2_train_key, dIdHate2_test_key):
    listItem.append(['log_{}'.format(test),
                     dIdHate2_train[train]['accuracy'],
                     dIdHate2_test[test]['accuracy'],
                     dIdHate2_train[train]['roc'],
                     dIdHate2_test[test]['roc'],
                     dIdHate2_train[train]['f1'],
                     dIdHate2_test[test]['f1'],
                     dIdHate2_train[train]['matthew'],
                     dIdHate2_test[test]['matthew'],
                     dIdHate2_train[train]['logloss'],
                     dIdHate2_test[test]['logloss'],])


dfIdHate2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHate2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.991522,0.991644,0.990169,0.958495,0.556584,0.563118,0.224729,0.207537,0.028775,0.031646
1,log_hasil_test_pp2_normal_ngram_2,0.991522,0.991644,0.990169,0.958495,0.556584,0.563118,0.224729,0.207537,0.028775,0.031646
2,log_hasil_test_pp2_normal_ngram_3,0.991522,0.991644,0.990169,0.958495,0.556584,0.563118,0.224729,0.207537,0.028775,0.031646
3,log_hasil_test_pp2_noCase_ngram_1,0.991683,0.991958,0.990295,0.964331,0.585754,0.609090,0.271298,0.292232,0.027104,0.029686
4,log_hasil_test_pp2_noCase_ngram_2,0.991683,0.991958,0.990295,0.964331,0.585754,0.609090,0.271298,0.292232,0.027104,0.029686
5,log_hasil_test_pp2_noCase_ngram_3,0.991683,0.991958,0.990295,0.964331,0.585754,0.609090,0.271298,0.292232,0.027104,0.029686
6,log_hasil_test_pp2_noPunc_ngram_1,0.991781,0.992188,0.991057,0.970698,0.588114,0.617955,0.284660,0.319590,0.026480,0.028853
7,log_hasil_test_pp2_noPunc_ngram_2,0.991781,0.992188,0.991057,0.970698,0.588114,0.617955,0.284660,0.319590,0.026480,0.028853
8,log_hasil_test_pp2_noPunc_ngram_3,0.991781,0.992188,0.991057,0.970698,0.588114,0.617955,0.284660,0.319590,0.026480,0.028853
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.992023,0.992417,0.990930,0.975695,0.625815,0.653692,0.339080,0.371868,0.024801,0.026936


## Clean

In [37]:
dClean_train = {}
dClean_test = {}
dClean2_train = {}
dClean2_test = {}

for item in ppKey:
    dClean_train['hasil_train_{}'.format(item)], dClean_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y7_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y7_{}'.format(item)])

for item2 in pp2Key:
    dClean2_train['hasil_train_{}'.format(item2)], dClean2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict2_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y7_{}'.format(item2)])

In [38]:
dClean_train_key = list(dClean_train.keys())
dClean_test_key = list(dClean_test.keys())
listItem = []

for train, test in zip(dClean_train_key, dClean_test_key):
    listItem.append(['log_{}'.format(test),
                     dClean_train[train]['accuracy'],
                     dClean_test[test]['accuracy'],
                     dClean_train[train]['roc'],
                     dClean_test[test]['roc'],
                     dClean_train[train]['f1'],
                     dClean_test[test]['f1'],
                     dClean_train[train]['matthew'],
                     dClean_test[test]['matthew'],
                     dClean_train[train]['logloss'],
                     dClean_test[test]['logloss'],])


dfCleanResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfCleanResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp_normal_ngram_1,0.956920,0.942179,0.982181,0.937006,0.857812,0.804679,0.740362,0.639039,0.113979,0.170317
1,log_hasil_test_pp_normal_ngram_2,0.956920,0.942179,0.982181,0.937006,0.857812,0.804679,0.740362,0.639039,0.113979,0.170317
2,log_hasil_test_pp_normal_ngram_3,0.956920,0.942179,0.982181,0.937006,0.857812,0.804679,0.740362,0.639039,0.113979,0.170317
3,log_hasil_test_pp_noCase_ngram_1,0.950546,0.942200,0.972346,0.938270,0.832377,0.802715,0.696253,0.638347,0.130031,0.169306
4,log_hasil_test_pp_noCase_ngram_2,0.950546,0.942200,0.972346,0.938270,0.832377,0.802715,0.696253,0.638347,0.130031,0.169306
5,log_hasil_test_pp_noCase_ngram_3,0.950546,0.942200,0.972346,0.938270,0.832377,0.802715,0.696253,0.638347,0.130031,0.169306
6,log_hasil_test_pp_noPunc_ngram_1,0.960026,0.949992,0.983485,0.950389,0.871326,0.838129,0.761192,0.695825,0.106650,0.155394
7,log_hasil_test_pp_noPunc_ngram_2,0.960026,0.949992,0.983485,0.950389,0.871326,0.838129,0.761192,0.695825,0.106650,0.155394
8,log_hasil_test_pp_noPunc_ngram_3,0.960026,0.949992,0.983485,0.950389,0.871326,0.838129,0.761192,0.695825,0.106650,0.155394
9,log_hasil_test_pp_noCasePunc_ngram_1,0.956499,0.950472,0.976591,0.952633,0.859081,0.839377,0.737801,0.698865,0.118181,0.153303


In [39]:
dClean2_train_key = list(dClean2_train.keys())
dClean2_test_key = list(dClean2_test.keys())
listItem = []

for train, test in zip(dClean2_train_key, dClean2_test_key):
    listItem.append(['log_{}'.format(test),
                     dClean2_train[train]['accuracy'],
                     dClean2_test[test]['accuracy'],
                     dClean2_train[train]['roc'],
                     dClean2_test[test]['roc'],
                     dClean2_train[train]['f1'],
                     dClean2_test[test]['f1'],
                     dClean2_train[train]['matthew'],
                     dClean2_test[test]['matthew'],
                     dClean2_train[train]['logloss'],
                     dClean2_test[test]['logloss'],])


dfClean2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfClean2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,log_hasil_test_pp2_normal_ngram_1,0.948191,0.942848,0.982910,0.949967,0.818061,0.799601,0.679604,0.641763,0.131436,0.155167
1,log_hasil_test_pp2_normal_ngram_2,0.948191,0.942848,0.982910,0.949967,0.818061,0.799601,0.679604,0.641763,0.131436,0.155167
2,log_hasil_test_pp2_normal_ngram_3,0.948191,0.942848,0.982910,0.949967,0.818061,0.799601,0.679604,0.641763,0.131436,0.155167
3,log_hasil_test_pp2_noCase_ngram_1,0.951011,0.946649,0.983729,0.954425,0.831636,0.817310,0.699620,0.669932,0.125301,0.147203
4,log_hasil_test_pp2_noCase_ngram_2,0.951011,0.946649,0.983729,0.954425,0.831636,0.817310,0.699620,0.669932,0.125301,0.147203
5,log_hasil_test_pp2_noCase_ngram_3,0.951011,0.946649,0.983729,0.954425,0.831636,0.817310,0.699620,0.669932,0.125301,0.147203
6,log_hasil_test_pp2_noPunc_ngram_1,0.955085,0.950556,0.986096,0.965053,0.849349,0.834560,0.727926,0.697925,0.116514,0.135150
7,log_hasil_test_pp2_noPunc_ngram_2,0.955085,0.950556,0.986096,0.965053,0.849349,0.834560,0.727926,0.697925,0.116514,0.135150
8,log_hasil_test_pp2_noPunc_ngram_3,0.955085,0.950556,0.986096,0.965053,0.849349,0.834560,0.727926,0.697925,0.116514,0.135150
9,log_hasil_test_pp2_noCasePunc_ngram_1,0.957967,0.954316,0.986421,0.967941,0.862051,0.850391,0.747422,0.724070,0.110274,0.126992


In [40]:
import pickle

In [43]:
dfToxicResult.to_csv('logisticRegression_toxic.csv',index=False)
dfToxic2Result.to_csv('logisticRegression_toxic2.csv',index=False)

dfSToxicResult.to_csv('logisticRegression_stoxic.csv',index=False)
dfSToxic2Result.to_csv('logisticRegression_stoxic2.csv',index=False)

dfObsceneResult.to_csv('logisticRegression_obscene.csv',index=False)
dfObscene2Result.to_csv('logisticRegression_obscene2.csv',index=False)

dfThreatResult.to_csv('logisticRegression_threat.csv',index=False)
dfThreat2Result.to_csv('logisticRegression_threat2.csv',index=False)

dfInsultResult.to_csv('logisticRegression_insult.csv',index=False)
dfInsult2Result.to_csv('logisticRegression_insult2.csv',index=False)

dfIdHateResult.to_csv('logisticRegression_idHate.csv',index=False)
dfIdHate2Result.to_csv('logisticRegression_idHate2.csv',index=False)

dfCleanResult.to_csv('logisticRegression_clean.csv',index=False)
dfClean2Result.to_csv('logisticRegression_clean2.csv',index=False)

In [42]:
model = '_log'

pickle.dump(dToxic_train, open('dToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic_test, open('dToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_train, open('dToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_test, open('dToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dSToxic_train, open('dSToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic_test, open('dSToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_train, open('dSToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_test, open('dSToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dObscene_train, open('dObscene_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene_test, open('dObscene_test{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_train, open('dObscene2_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_test, open('dObscene2_test{}.sav'.format(model), 'wb'))

pickle.dump(dThreat_train, open('dThreat_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat_test, open('dThreat_test{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_train, open('dThreat2_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_test, open('dThreat2_test{}.sav'.format(model), 'wb'))

pickle.dump(dInsult_train, open('dInsult_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult_test, open('dInsult_test{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_train, open('dInsult2_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_test, open('dInsult2_test{}.sav'.format(model), 'wb'))

pickle.dump(dIdHate_train, open('dIdHate_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate_test, open('dIdHate_test{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_train, open('dIdHate2_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_test, open('dIdHate2_test{}.sav'.format(model), 'wb'))

pickle.dump(dClean_train, open('dClean_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean_test, open('dClean_test{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_train, open('dClean2_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_test, open('dClean2_test{}.sav'.format(model), 'wb'))

In [44]:
pickle.dump(dPredict_train, open('dPredict_train.sav', 'wb'))
pickle.dump(dPredict_test, open('dPredict_test.sav', 'wb'))
pickle.dump(dPredictProba_train, open('dPredictProba_train.sav', 'wb'))
pickle.dump(dPredictProba_test, open('dPredictProba_test.sav', 'wb'))

pickle.dump(dPredict2_train, open('dPredict2_train.sav', 'wb'))
pickle.dump(dPredict2_test, open('dPredict2_test.sav', 'wb'))
pickle.dump(dPredict2Proba_train, open('dPredict2Proba_train.sav', 'wb'))
pickle.dump(dPredict2Proba_test, open('dPredict2Proba_test.sav', 'wb'))

In [45]:
X_test

64748     "  The point I think was very simply about bia...
10958     Please refrain from removing content from Wiki...
114719    See Tenth Amendment to the United States Const...
95438     "  Note that the warning was due to the fact t...
63391     See also User_talk:Webapp where I was blocked ...
22070     @ Samaleks - I have seen your allegation of SO...
121506    That official BBC link refers to it as series ...
125814    "   I am a WP:SPI clerk, so it is my job to wo...
101745    Soccer is only required to be used once per ar...
65758                   REDIRECT Talk:Donna Moss (designer)
11628     "   Your edits to Rabbit   Please stop replaci...
154068    Lying, scheming, vindictive Weasel   Burn in h...
25959     "  You know, don't be so sure of ""obviously f...
143816    " (outdent) Please don't feed the trolls by re...
112905    " which is something you are supposed to do as...
48585     "  Liberals  Btw Dave, ""Liberal"" means somth...
63697     The Santana   Hi, I'm starting